In [ ]:
## A system which interferes if a student is about to fail. Classifcation because output 1 (intervention is required)
## 0 (intervention is not required) Output is discrete not continuous

In [ ]:
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score ## It is harmonic mean of both precision and recall

student_data = pd.read_csv("student-data.csv")

In [ ]:
student_data.head()  ## prints 1st 5 rows

In [ ]:
student_data.shape  ## not.shape()

In [ ]:
n_students = student_data.shape[0]  ## .shape IMp 
n_features = student_data.shape[1] - 1 ## -1 imp to remove output label

passed = student_data.loc[student_data.passed == "yes","passed"] ## .loc searches with labels while iloc searches with location
n_passed = passed.shape[0]

failed = student_data.loc[student_data.passed == "no","passed"]
n_failed = failed.shape[0]

total = float(n_passed + n_failed) ## Typecating to float
grad_rate = float((n_passed *100)/total)

print("Data Exploration--->")
print("Total Students\t{}".format(n_students))
print("Total Passed\t{}".format(n_passed))
print("Total Failed\t{}".format(n_failed))
print("Graduation rate\t{:.2f}%".format(grad_rate))

In [ ]:
feature_cols = list(student_data.columns[:-1]) ##.columns yileds column names in array format list() converts them to list fromat
target_col = student_data.columns[-1] ## Acces last index in columns array i.e y

print("Features Colums\n{}".format(feature_cols))
print("Target Column\n{}".format(target_col))

X_all = student_data[feature_cols]
Y_all = student_data[target_col]

X_all.head()

In [ ]:
### Preprocessing--->Convert yes/no to 1/0
## variables having more than to categories are fed to pd.get_dummies()
def preprocess_features(X):
    
    output = pd.DataFrame(index=X.index) ## Creates an empty dataframe with index same as data
    
    for col, col_data in X.iteritems(): ## Iterating over columns---
        
        if col_data.dtype == object:  ## if data has dtype of non-numerical
            col_data = col_data.replace(['yes','no'],[1,0])  ## replacing yes/no to 1/0
            
        if col_data.dtype == object:  ## if data has dtype of categorical
            ### Example school is converted to school_gp, school_ms
            col_data = pd.get_dummies(col_data, prefix=col)  
        
        output = output.join(col_data)
        
    return output

X_all = preprocess_features(X_all)
print("Processed feature columns ({} total columns):\t{}".format(len(X_all.columns),list(X_all.columns)))

In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_all, Y_all, stratify = Y_all, test_size=95, random_state=42)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

In [ ]:
## Since this is classification we are going to use
## 1. Naive Bayes--> It performs well on small dataset
## 2. logistic Regression --> Since there is a lot of correlation among features, also we could regularization ter,
## 3. SVM
## Then choice best of above

In [ ]:
def train_classifier(clf,X_train,Y_train): ## Function to train classifier
    
    start = time()
    clf.fit(X_train,Y_train)
    end = time()
    
    print(" Model trained in {:.4f} seconds".format(end-start))

In [ ]:
def predict_labels(clf,features,target): ## Model to predict labels can be for training/testing both
    
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    print("Model predicted value in {:.4f}seconds".format(end-start))
    
    return f1_score(target.values,y_pred,pos_label="yes")

In [ ]:
def train_predict(clf, X_train, y_train, X_test, y_test):

    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    train_classifier(clf, X_train, y_train)
    
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.\n".format(predict_labels(clf, X_test, y_test)))

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

clf_A = GaussianNB()
clf_B = LogisticRegression(random_state=42)
clf_C = SVC(random_state=42)

X_train_100 = X_train.iloc[:100, :] ## i.e 1st using top 100 data
Y_train_100 = Y_train.iloc[:100]

X_train_200 = X_train.iloc[:200, :]
Y_train_200 = Y_train.iloc[:200]

X_train_300 = X_train.iloc[:300, :]
Y_train_300 = Y_train.iloc[:300]

for clf in [clf_A, clf_B, clf_C]:
    print ("\n{}: \n\n".format(clf.__class__.__name__))
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], Y_train[:n], X_test, Y_test)

In [30]:
## We can clearly see the results are much better in SVM
### BUT SvM has O(n^3) hence once the data increses the computation time would increse tremendously 
### Hence we will choice logistic regression

## GridSearchCV ---> In this method we exhaustively search over a given a set of values for a given hyperparametr 
## finding the best one

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit

C = [0.001, 0.01, 0.1, 1, 10, 100, 1000] ## Different values of C in LIST!!!
solver = ['sag']
max_iter = [1000]
param_grid = dict(C=C, solver=solver, max_iter=max_iter)

clf = LogisticRegression(random_state=42) ## Our classifer 

f1_scorer = make_scorer(f1_score, pos_label='yes') ## Thsi is the scoring function
##We will use a stratified shuffle split data-split since it preserves the percentage of samples 
## for each class and combines it with cross validation
ssscv = StratifiedShuffleSplit(Y_train, n_iter=10, test_size=0.1)  

## CV parameter is our cross validation generator
grid_obj = GridSearchCV(clf, param_grid, cv=ssscv, scoring=f1_scorer) ### Scoring--> how we have to measure

grid_obj = grid_obj.fit(X_train, Y_train) ## Fitting the data

clf = grid_obj.best_estimator_ ## FInding the best parameters

# Report the final F1 score for training and testing after parameter tuning
print ("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, Y_train)))
print ("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, Y_test)))